## Data Ingestion

In [4]:
import os
%pwd

'd:\\Data Science\\END to END Proj\\WinePred\\winepred-MLFLOW'

In [3]:
os.chdir("../")

In [5]:
## ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from src.WinePredMLflow.constant import *
from src.WinePredMLflow.utils.common import read_yaml,create_directories 

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from WinePredMLflow import logger
from WinePredMLflow.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the file from source_URL to local_data_file"""
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
        
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_URL}...")
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,  # Use source_URL here
                filename=self.config.local_data_file  # Save to local_data_file
            )
            logger.info(f"Download completed to: {filename}")
            logger.debug(f"Download headers: {headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}, size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """Extracts the downloaded zip file to unzip_dir"""
        logger.info(f"Extracting zip file from {self.config.local_data_file} to {self.config.unzip_dir}")
        
        # Create extraction directory if it doesn't exist
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        
        logger.info(f"Successfully extracted to {self.config.unzip_dir}")

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-27 14:45:03,005: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-27 14:45:03,010: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-27 14:45:03,017: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-27 14:45:03,025: INFO: common: created directory at: artifacts]
[2025-06-27 14:45:03,027: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-27 14:45:03,027: INFO: 2235201259: File already exists at artifacts/data_ingestion/data.zip, size: ~ 23 KB]
[2025-06-27 14:45:03,033: INFO: 2235201259: Extracting zip file from artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
[2025-06-27 14:45:03,045: INFO: 2235201259: Successfully extracted to artifacts/data_ingestion]
